## Matching Datasets

Things that I've learned:

- The Excels match the un-updated PDFs.

- Example: 
    - 1.9 Federal Transitional Reinsurance Program payments expected from HHS (as indicated by HHS as of 6/30). For Aetna Health of FL: 10,355,176.30.
    - Reported in the original PDF: 18628 Aetna Health Inc. (a FL corp.) FL 10,355,176.30 (28,025,200.75) 5,437,975.83
    - Reported in the updated PDF: 18628 Aetna Health Inc. (a FL corp.) FL 10,360,565.46 (28,025,200.75) 5,437,975.83

My goal with this Python notebook is to create final spreadsheets for each year. I imagine the final spreadsheet will be like:

HHOS ID ... MR_SUBMISSION_TEMPLATE_ID ... COMPANY NAME ... REPORTED VALUES ... ACTUAL VALUES

for 2014 and 2015. That way, we can do our time analysis.

In [166]:
import pandas as pd
import numpy as np

# for debugging, display everything: 
pd.options.display.max_seq_items = 2000
pd.options.display.max_rows = 4000
pd.options.mode.chained_assignment = None  # default='warn'

In [122]:
from re import sub
from decimal import Decimal

# strip the pdf data to make it parseable

def strip_money_values( column ):
    i = 0
    for money in pdf_data[column]:
        if money == '$-' or money == 'N/E' or \
            money == 'N/A_MA_Issuer' or money == 'N/A_DefaultCharge' or \
            money == 'N/A_Default_Charge':
            pdf_data[column][i] = "N/A"
        else:
            value = Decimal(sub(r'[^\d.]', '', money))
            if '(' in money:
                value = -1 * value
            else:
                value = value
            pdf_data[column][i] = value
        i = i + 1

print("strip_money_values defined")

def perform_stripping( path ): 
    columns = ["HHS RISK ADJUSTMENT TRANSFER AMOUNT (INDIVIDUAL MARKET, INCLUDING CATASTROPHIC)",
            "REINSURANCE PAYMENT AMOUNT (OR NOT ELIGIBLE)",
            "HHS RISK ADJUSTMENT TRANSFERS AMOUNT (SMALL GROUP MARKET)"]

    for column in columns:
        strip_money_values(column)
    print(pdf_data)
    pdf_data.to_csv(path_or_buf=path)
    

print("perform_stripping defined")

strip_money_values defined
perform_stripping defined


In [123]:
# how to execute the above scripts to clean the data of all '$', ',', and '(')''s.
#pdf_data = pd.read_csv("input/2014-Benefit-Year-RI-RA-Updated.csv")
#perform_stripping("input/2014-Benefit-Year-RI-RA-Updated-numeric.csv")
# THIS CREATES THE FOLLOWING SPREADSHEET:
# HIOS ID ... COMPANY NAME ... STATE ... REINSURANCE ... RISK ADJUSTMENT INDIDIVDUAL ... RISK ADJUSTMENT SMALL GROUP

In [167]:
# WE WANT TO CREATE THE FOLLOWING SPREADSHEET:
# SUBMISSION ID ... MEMBER MONTHS ... REINSURANCE ... RISK ADJUSTMENT INDIVIDUAL ... RISK ADJUSTMENT SMALL GROUP

# read in excel data
excel_data_2015 = pd.read_csv("input/2015_Part1_2_Summary_Data_Premium_Claims.csv", dtype=str)
excel_data_2015 = excel_data_2015.apply(pd.to_numeric, errors='ignore')

# only keep the rows that we are looking up
a_data_2015 = excel_data_2015.loc[excel_data_2015["ROW_LOOKUP_CODE"]
                            .isin({"FED_REINS_PAYMENTS", "FED_RISK_ADJ_NET_PAYMENTS", "MEMBER_MONTHS"})]

# only keep the columns that contain numbers that we are looking up
b_data_2015 = a_data_2015[["MR_SUBMISSION_TEMPLATE_ID", "ROW_LOOKUP_CODE", "CMM_INDIVIDUAL_Q1", "CMM_SMALL_GROUP_Q1"]]

# remove rows that contain null values in *BOTH* columns that we are looking up
# (thus, threshold = 3 null columns to drop the column)
c_data_2015 = b_data_2015.dropna(thresh=3)

# drop duplicates in the CMM_INDIVIDUAL_Q1 and CMM_SMALL_GROUP_Q1 rows. 
# for some reason, companies are reported twice.
d_data_2015 = c_data_2015.drop_duplicates(subset=["CMM_INDIVIDUAL_Q1","CMM_SMALL_GROUP_Q1"])

# get member months data as base for combining the rows
df = d_data_2015.loc[d_data_2015["ROW_LOOKUP_CODE"].isin({"MEMBER_MONTHS"})]
df = df[["MR_SUBMISSION_TEMPLATE_ID","CMM_INDIVIDUAL_Q1","CMM_SMALL_GROUP_Q1"]]
df["MEMBER_MONTHS_INDIVIDUAL"] = df["CMM_INDIVIDUAL_Q1"]
df["MEMBER_MONTHS_SMALL_GROUP"] = df["CMM_SMALL_GROUP_Q1"]
del df["CMM_INDIVIDUAL_Q1"]
del df["CMM_SMALL_GROUP_Q1"]

# merge the reinsurance
data_2015_reins = d_data_2015.loc[d_data_2015["ROW_LOOKUP_CODE"].isin({"FED_REINS_PAYMENTS"})]

# rename column to reinsurance
data_2015_reins["REINSURANCE"] = data_2015_reins["CMM_INDIVIDUAL_Q1"]
data_2015_reins = data_2015_reins[["MR_SUBMISSION_TEMPLATE_ID","REINSURANCE"]]
df = df.merge(data_2015_reins, on="MR_SUBMISSION_TEMPLATE_ID")

# merge the risk adjustment
data_2015_riskadj = d_data_2015.loc[d_data_2015["ROW_LOOKUP_CODE"].isin({"FED_RISK_ADJ_NET_PAYMENTS"})]

# rename columns for merging
data_2015_riskadj["RISK_TRANSFER_INDIVIDUAL"] = data_2015_riskadj["CMM_INDIVIDUAL_Q1"]
data_2015_riskadj["RISK_TRANSFER_SMALL_GROUP"] = data_2015_riskadj["CMM_SMALL_GROUP_Q1"]
data_2015_riskadj = data_2015_riskadj[["MR_SUBMISSION_TEMPLATE_ID","RISK_TRANSFER_INDIVIDUAL","RISK_TRANSFER_SMALL_GROUP"]]
df = df.merge(data_2015_riskadj, on="MR_SUBMISSION_TEMPLATE_ID")

df[np.isnan(df)] = 0

In [168]:
# now we have a file like this. how do we match it?
# 1) if reinsurance, risk_transfer_individual, or risk_transfer_small_group match exactly, then there's a match.
# 2) find the closest match otherwise.
df["Hash"] = df["REINSURANCE"] * 7 + df["RISK_TRANSFER_INDIVIDUAL"] * 31 + df["RISK_TRANSFER_SMALL_GROUP"] * 17
df = df.sort_values(by="Hash")
print(df.head())
print(df.tail())

     MR_SUBMISSION_TEMPLATE_ID  MEMBER_MONTHS_INDIVIDUAL  \
143                     142009                 1722261.0   
105                     140839                 2736915.0   
139                     141992                 3470273.0   
88                      140366                 1358859.0   
267                     143689                 7778647.0   

     MEMBER_MONTHS_SMALL_GROUP   REINSURANCE  RISK_TRANSFER_INDIVIDUAL  \
143                        0.0  1.901151e+07              -218903904.1   
105                  1788573.0  4.187840e+07              -125635856.0   
139                   851617.0  1.367631e+08              -135056370.8   
88                    681644.0  5.173671e+07               -37496765.0   
267                  8405443.0  1.989800e+08               -97545960.0   

     RISK_TRANSFER_SMALL_GROUP          Hash  
143               0.000000e+00 -6.652940e+09  
105              -9.524829e+06 -3.763485e+09  
139              -1.151183e+07 -3.425107e+09  
88    

In [180]:
df2 = pd.read_csv("input/2015-Benefit-Year-RI-RA-Not-Updated-numeric.csv")
df2 = df2.rename(columns={'REINSURANCE PAYMENT AMOUNT (OR NOT ELIGIBLE)': 'REINSURANCE',
     'HHS RISK ADJUSTMENT TRANSFER AMOUNT (INDIVIDUAL MARKET, INCLUDING CATASTROPHIC)': 'RISK_TRANSFER_INDIVIDUAL',
     'HHS RISK ADJUSTMENT TRANSFERS AMOUNT (SMALL GROUP MARKET)':'RISK_TRANSFER_SMALL_GROUP'})
df2 = df2.apply(pd.to_numeric, errors='ignore')

try:
    del df2["Unnamed: 0"]
except KeyError:
    pass

try: 
    del df2["Unnamed: 0.1"]
except KeyError:
    pass

df2.fillna(0)
print(df2)
#df2["REINSURANCE"][np.isnan(df["REINSURANCE"])] = 0
#df2["RISK_TRANSFER_INDIVIDUAL"][np.isnan(df["RISK_TRANSFER_INDIVIDUAL"])] = 0
#df2["RISK_TRANSFER_SMALL_GROUP"][np.isnan(df["RISK_TRANSFER_SMALL_GROUP"])] = 0

df2["Hash"] = df2["REINSURANCE"] * 7 + df2["RISK_TRANSFER_INDIVIDUAL"] * 31 + df2["RISK_TRANSFER_SMALL_GROUP"] * 17
df2 = df2.sort_values(by="Hash")

#print(df)
df2 = df2.merge(df, on="Hash", how='left')
print(df2.head())
print(df2.tail())
#print(df.dtypes)

    HIOS ID               HIOS INPUTTED INSURANCE COMPANY NAME STATE  \
0     11082                       Aetna Life Insurance Company    AK   
1     38344           Premera Blue Cross Blue Shield of Alaska    AK   
2     38596                             Time Insurance Company    AK   
3     62637                  John Alden Life Insurance Company    AK   
4     73836                             Moda Health Plan, Inc.    AK   
5     80049                United Healthcare Insurance Company    AK   
6     13939                             Time Insurance Company    AL   
7     44580                           Humana Insurance Company    AL   
8     44902                 Federated Mutual Insurance Company    AL   
9     46944              Blue Cross and Blue Shield of Alabama    AL   
10    49771                  John Alden Life Insurance Company    AL   
11    59809           United Healthcare Life Insurance Company    AL   
12    68259                 United Healthcare of Alabama, Inc.  